<a href="https://colab.research.google.com/github/safal207/hello-world/blob/master/dl_nlp_yelp_polarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Определение тональности текстов на основе отзывов Yelp с использованием сети LSTM

В этом ноутбуке мы построим и обучим рекуррентную нейронную сеть (RNN) для классификации отзывов на два класса: положительные и отрицательные. Мы будем использовать набор данных [Yelp Review Polarity](https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz), который содержит 560,000 обучающих и 38,000 тестовых отзывов.

**План работы:**
1.  **Загрузка данных:** Скачаем и распакуем набор данных Yelp.
2.  **Предобработка текста:** Преобразуем текстовые отзывы в числовой формат, понятный для нейронной сети, с помощью токенизации и дополнения последовательностей.
3.  **Создание модели:** Спроектируем нейронную сеть с использованием слоев `Embedding` и `LSTM`.
4.  **Обучение модели:** Обучим модель на обучающем наборе данных и будем отслеживать ее производительность.
5.  **Оценка качества:** Оценим точность обученной модели на тестовом наборе данных.
6.  **Прогнозирование:** Используем модель для предсказания тональности нового отзыва.

### 1. Подготовка и настройка

In [ ]:
%tensorflow_version 2.x
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

%matplotlib inline

In [ ]:
# Гиперпараметры
num_words = 10000  # Размер словаря (количество наиболее частых слов)
max_review_len = 100  # Максимальная длина отзыва в словах

### 2. Загрузка набора данных

Используем команду `wget` для загрузки архива с данными из облачного хранилища и `tar` для его распаковки.

In [ ]:
!wget https://www.dropbox.com/s/ufbhk3kadtnn6h0/yelp_review_polarity_csv.tgz?dl=1 -O yelp_review_polarity_csv.tgz

In [ ]:
!tar -xvf yelp_review_polarity_csv.tgz

#### Просмотр данных
Давайте посмотрим на описание набора данных и примеры из обучающего и тестового файлов.

In [ ]:
!cat yelp_review_polarity_csv/readme.txt

In [ ]:
!head yelp_review_polarity_csv/train.csv

In [ ]:
!wc -l yelp_review_polarity_csv/train.csv
!wc -l yelp_review_polarity_csv/test.csv

### 3. Предобработка данных
Загружаем данные из CSV-файлов в DataFrame-ы pandas.

In [ ]:
train = pd.read_csv(
    "yelp_review_polarity_csv/train.csv", header=None, names=["Class", "Review"]
)

In [ ]:
train.head()

Выделяем данные для обучения: тексты отзывов и их классы (тональность).

In [ ]:
reviews = train["Review"]

In [ ]:
# Классы в датасете: 1 (негативный) и 2 (позитивный). Преобразуем их в 0 и 1.
y_train = train["Class"] - 1

#### Токенизация текста
Нейронные сети не могут работать с текстом напрямую. Нам нужно преобразовать текст в числа. Этот процесс называется **токенизацией**.

Мы выполним следующие шаги:
1.  **Создадим словарь:** Используем `Tokenizer` из Keras для анализа всех текстов и создания словаря, где каждому уникальному слову присваивается свой номер (индекс).
2.  **Преобразуем тексты в последовательности:** Заменим каждое слово в отзывах на его индекс из словаря.
3.  **Дополним последовательности:** Так как отзывы имеют разную длину, мы приведем их все к одной длине (`max_review_len`), добавляя нули в начало коротких отзывов или обрезая длинные. Этот процесс называется **padding**.

In [ ]:
# Создаем токенизатор Keras, который будет учитывать только 10,000 самых частых слов
tokenizer = Tokenizer(num_words=num_words)

In [ ]:
# Обучаем токенизатор на наших данных
tokenizer.fit_on_texts(reviews)

In [ ]:
# Преобразуем тексты в числовые последовательности
sequences = tokenizer.texts_to_sequences(reviews)

Пример преобразования отзыва в последовательность чисел:

In [ ]:
index = 1
print(reviews[index])
print(sequences[index])

In [ ]:
# Дополняем последовательности до фиксированной длины
x_train = pad_sequences(sequences, maxlen=max_review_len)

### 4. Создание и обучение нейронной сети

Наша модель будет состоять из трех слоев:
1.  **Embedding:** Этот слой преобразует числовые индексы слов в плотные векторы фиксированной длины (64). Он позволяет сети улавливать семантические связи между словами.
2.  **LSTM (Long Short-Term Memory):** Это рекуррентный слой, который хорошо подходит для обработки последовательных данных, таких как текст. Он может запоминать информацию на длинных дистанциях, что важно для понимания контекста в отзывах.
3.  **Dense:** Стандартный полносвязный слой с одним нейроном и `sigmoid` активацией, который будет выдавать на выходе вероятность от 0 до 1, показывающую, является ли отзыв положительным.

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 64, input_length=max_review_len))
model.add(LSTM(128))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Создаем `callback` для сохранения лучшей версии модели (по метрике `val_accuracy`) во время обучения.

In [ ]:
model_save_path = "best_model.h5"
checkpoint_callback = ModelCheckpoint(
    model_save_path,
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1,
)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.1,
    callbacks=[checkpoint_callback],
)

In [ ]:
# Отрисовываем график результатов обучения
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(history.history['accuracy'], label='Точность на обучающем наборе')
ax.plot(history.history['val_accuracy'], label='Точность на проверочном наборе')
ax.set_xlabel('Эпоха обучения')
ax.set_ylabel('Точность')
ax.set_title('График точности модели')
ax.legend()
plt.grid(True)
plt.show()

### 5. Оценка качества на тестовых данных

In [ ]:
model.load_weights(model_save_path)

In [ ]:
test = pd.read_csv(
    "yelp_review_polarity_csv/test.csv", header=None, names=["Class", "Review"]
)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test["Review"])

In [ ]:
x_test = pad_sequences(test_sequences, maxlen=max_review_len)

In [ ]:
y_test = test["Class"] - 1

In [ ]:
model.evaluate(x_test, y_test, verbose=1)

### 6. Прогнозирование на новом отзыве

In [ ]:
text = '''The SmartBurger restaurant is awful. It’s a small shabby place. 
The food is really bad and very expensive.  The host and waiters are rude. 
I will never visit the SmartBurger again!
'''

In [ ]:
sequence = tokenizer.texts_to_sequences([text])

In [ ]:
data = pad_sequences(sequence, maxlen=max_review_len)

In [ ]:
result = model.predict(data)

In [ ]:
if result[[0]] < 0.5:
    print("Отзыв отрицательный")
else:
    print("Отзыв положительный")

### 7. Анализ ошибок модели

Чтобы лучше понять, где модель ошибается, давайте посмотрим на несколько отзывов, которые были классифицированы неверно.

In [ ]:
# Получаем предсказания для всего тестового набора
predictions = (model.predict(x_test) > 0.5).astype("int32")

# Находим индексы, где предсказания не совпадают с истинными метками
y_test_flat = y_test.to_numpy()
misclassified_indices = np.where(predictions.flatten() != y_test_flat)[0]

print(f"Количество неверно классифицированных отзывов: {len(misclassified_indices)}")

# Выводим несколько примеров
print("\nПримеры неверно классифицированных отзывов:\n")
for i, index in enumerate(misclassified_indices[:5]):
    review_text = test.iloc[index]['Review']
    true_label = "Позитивный" if y_test.iloc[index] == 1 else "Негативный"
    predicted_label = "Позитивный" if predictions[index][0] == 1 else "Негативный"
    
    print(f"--- Пример {i+1} ---")
    print(f"Текст: {review_text[:500]}...")
    print(f"Истинная метка: {true_label}")
    print(f"Предсказанная метка: {predicted_label}")
    print("-" * (15 + len(str(i+1))))

## Задания для самостоятельной работы

1. Используйте сеть GRU вместо LSTM для определения тональности отзывов Yelp. Сравните скорость обучения и качество работы обученной сети.
2. Меняйте гиперпараметры нейросети, чтобы повысить качество работы:
  - Длину вектора представления слов в слое Embedding.
  - Количество нейронов на рекуррентном слое (LSTM или GRU).
  - Количество рекуррентные слоев.
  - Тип оптимизатора (`adam`, `rmsprop` и др.)
  - Количество эпох обучения.
  - Разрмер мини-выборки.
3.Попробуйте применить вместо токенизатора Keras более совершенные:
  - [spaCy Tokenizer](https://spacy.io/usage/spacy-101#annotations-token)
  - [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
  
    Эти токенизаторы сложнее в использовании, но работают лучше.
4. Переделайте нейросеть для распознавания набора данных [Yelp reviews - Full](https://s3.amazonaws.com/fast-ai-nlp/yelp_review_full_csv.tgz), в котором 4 класса по количеству звезд у отзывов.